In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
# ! git pull

## Add import path

In [ ]:
import gc
import os
import sys

In [ ]:
def add_library_level(level=4):
    suf_path = ['..']
    path = '..'
    for i in range(0, level):
        join_path = suf_path * i
        path = '/'.join(join_path)
        module_path = os.path.abspath(os.path.join(path))
        if module_path not in sys.path:
            sys.path.append(module_path)
            print(f'Appendeding {path}')

In [ ]:
add_library_level(level=4)

## Organize imports

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from torch import nn

In [ ]:
from sae_lens import SAE, HookedSAETransformer
from transformer_lens.utils import tokenize_and_concatenate

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
from datasets import load_dataset

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from tqdm import tqdm

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
import plotly.express as px

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

In [ ]:
from src.lattmc.sae.nlp_sae_utils import *

In [ ]:
import logging

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
device = init_device()

In [ ]:
torch.__version__

In [ ]:
np.__version__

## Initialize Layer

In [ ]:
layer = 4
dataset_suffix = 'pile'

## Initialize Path

In [ ]:
def init_data(model_name, layer, dataset_suffix='pile'):
    PATH = Path('../data')
    dataset_suffix = 'pile'
    vectors_name = f'{model_name.replace('-', '_')}_{layer}_{dataset_suffix}'
    checkpoint_dir = PATH / 'saes'
    vectors_dir = checkpoint_dir / f'{vectors_name}_vecs'
    matrix_dir = checkpoint_dir / f'{vectors_name}_mats'
    vectors_path = checkpoint_dir / f'{vectors_name}_vecs.joblib'
    
    mkdirs(
        checkpoint_dir,
        vectors_dir,
        matrix_dir
    )
    logger.info(f'{matrix_dir = } {vectors_path = }')

    return matrix_dir, vectors_path

## Initialize simple dataset

In [ ]:
dataset = load_dataset(
    path='NeelNanda/pile-10k',
    split='train',
    streaming=False,
)

## Initialize model

In [ ]:
def gen_concept(idx, val, shape=24576):
    v_idx = np.zeros((shape,), dtype=float)
    v_idx[idx] = val
    concept = fca.G_FG(v_idx)
    
    return v_idx, concept

In [ ]:
model_name = 'gpt2-small'
release = 'gpt2-small-res-jb'
sae_id = f'blocks.{layer}.hook_resid_pre'

In [ ]:
matrix_dir, vector_path = init_data(model_name, layer, dataset_suffix=dataset_suffix)

In [ ]:
net = Text2Sae(
    model_name,
    release,
    sae_id,
    device
)

## Generate V Lattice

In [ ]:
gc.collect()

In [ ]:
matrix_dir

In [ ]:
init_matrices(matrix_dir, dataset, net)

In [ ]:
gc.collect()

In [ ]:
V = init_vectors(vector_path, matrix_dir, segment=False)

## Generate Context and Analyze

In [ ]:
gc.collect()

In [ ]:
fca = FCA(V)

In [ ]:
text1 = "The Golden Gate Bridge"
text2 = "The Brooklyn Bridge"
text3 = "The card game Bridge"

In [ ]:
sent1 = 'New York City'
sent2 = 'Golden Gate Bridge'
sent3 = 'Grand Canyon Park'

In [ ]:
word_mapper = WordMapper(net, dataset, matrix_dir)

## New York Ciry

In [ ]:
v_ns = net.embed(sent1)[0][1:]
v_gs = net.embed(sent2)[0][1:]

In [ ]:
t_ns = net.tokenize(sent1)[0][1:]
t_gs = net.tokenize(sent2)[0][1:]

In [ ]:
t_ns.shape, t_gs.shape

In [ ]:
t_ns, t_gs

In [ ]:
topK(v_ns[0], 10)

In [ ]:
topK(v_gs[0], 10)

In [ ]:
m_bg = meet(v_ns[2], v_gs[2])
topK(m_bg, 20)

In [ ]:
m_cg, c_bg = gen_concept(9805, 6)
c_bg

In [ ]:
ws = word_mapper(c_bg, v=m_cg)

In [ ]:
ws

## Sentence generation

In [ ]:
c_golden = fca.G_FG(v_gs[0] / 10)
c_gate = fca.G_FG(v_gs[1] / 10)
c_bridge = fca.G_FG(v_gs[2] / 10)

In [ ]:
c_golden, c_gate, c_bridge

In [ ]:
c_golden_gate = c_golden & c_gate
c_golden_gate

In [ ]:
dataset[c_golden_gate.A[0].item()]['text']

In [ ]:
v_nyc = net.embed(sent1)[0][1:]
v_ggb = net.embed(sent2)[0][1:]
v_gcp = net.embed(sent3)[0][1:]
v_nyc.shape, sent1, v_ggb.shape, sent2, v_gcp.shape, sent3

In [ ]:
join_all(v_ggb).shape

In [ ]:
c_ggb = fca.G_FG(v_ggb[1] / 10)
c_ggb

In [ ]:
dataset[c_ggb.A[2].item()]['text']

In [ ]:
ws = word_mapper(c_ggb, v=v_ggb[0] / 100)

In [ ]:
ws

In [ ]:
topK(v_gcp[0], 20)

In [ ]:
v_gcm = meet(v_gcp[0], v_gcp[1])
topK(v_gcm, 10)

In [ ]:
v_ggm = meet(v_ggb[0], v_ggb[1])
topK(v_ggm, 10)

In [ ]:
c_ggm = fca.G_FG(v_ggm)

In [ ]:
w_ggm = word_mapper(c_ggm)
w_ggm

In [ ]:
items = dataset[c_ggm.A.tolist()]

In [ ]:
items

In [ ]:
v_i, cn_i = gen_concept(6374, 6)
cn_i

In [ ]:
ws = word_mapper(cn_i, v=v_i)

In [ ]:
ws